In [1]:
import sys

sys.path.append('..')
import os
import torch
import pandas as pd
from deep_logic.models.relu_nn import XReluNN
from deep_logic.models.psi_nn import PsiNetwork
from deep_logic.models.tree import XDecisionTreeClassifier
from deep_logic.utils.base import set_seed
from deep_logic.utils.metrics import F1Score
from deep_logic.models.general_nn import XGeneralNN
from deep_logic.utils.datasets import ConceptToTaskDataset
from deep_logic.utils.data import get_splits_train_val_test
from deep_logic.logic.base import test_multi_class_explanation

results_dir = 'results/cub'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

## Define loss, metrics and saved metrics

In [2]:
loss = torch.nn.CrossEntropyLoss()
metric = F1Score()

methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []

## Loading CUB data

In [3]:
dataset = ConceptToTaskDataset("../data/CUB_200_2011/")
concept_names = dataset.attribute_names
print("Concept names", concept_names)

class_to_explain = [0, 1]
class_to_explain_names = [dataset.classes[class_to_explain[0]],
                          dataset.classes[class_to_explain[1]]]
print("Class names", class_to_explain_names)
n_features = dataset.n_attributes
print("Number of features", n_features)

Concept names ['has_bill_shape_dagger' 'has_bill_shape_hooked_seabird'
 'has_bill_shape_allpurpose' 'has_bill_shape_cone' 'has_wing_color_brown'
 'has_wing_color_grey' 'has_wing_color_yellow' 'has_wing_color_black'
 'has_wing_color_white' 'has_wing_color_buff' 'has_upperparts_color_brown'
 'has_upperparts_color_grey' 'has_upperparts_color_yellow'
 'has_upperparts_color_black' 'has_upperparts_color_white'
 'has_upperparts_color_buff' 'has_underparts_color_brown'
 'has_underparts_color_grey' 'has_underparts_color_yellow'
 'has_underparts_color_black' 'has_underparts_color_white'
 'has_underparts_color_buff' 'has_breast_pattern_solid'
 'has_breast_pattern_striped' 'has_breast_pattern_multicolored'
 'has_back_color_brown' 'has_back_color_grey' 'has_back_color_yellow'
 'has_back_color_black' 'has_back_color_white' 'has_back_color_buff'
 'has_tail_shape_notched_tail' 'has_upper_tail_color_brown'
 'has_upper_tail_color_grey' 'has_upper_tail_color_black'
 'has_upper_tail_color_white' 'has_uppe

## Training Hyperparameters

In [4]:
epochs = 200
l_r = 0.001
lr_scheduler = True
top_k_explanations = 1
simplify = True
seeds = [*range(10)]
print("Seeds", seeds)

Seeds [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Decision Tree

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    device = torch.device("cpu")

    print("Training Tree Classifier...")
    model = XDecisionTreeClassifier(n_classes=dataset.n_classes, n_features=n_features)

    results = model.fit(train_data, val_data, metric=metric, save=False)
    print(results)
    accuracy = model.evaluate(test_data)
    print("Test model accuracy", accuracy)

    formula, elapsed_time = model.get_global_explanation(class_to_explain[0], concept_names, return_time=True)
    print(f"{class_to_explain_names[0]} <-> {formula}")
    print("Elapsed time", elapsed_time)

    formula_2, elapsed_time_2 = model.get_global_explanation(class_to_explain[1], concept_names, return_time=True)
    print(f"{class_to_explain_names[1]} <-> {formula_2}")
    print("Elapsed time", elapsed_time_2)

    methods.append("Tree")
    splits.append(seed)
    explanations.append(formula)
    explanations_inv.append(formula_2)
    model_accuracies.append(accuracy)
    explanation_accuracies.append(accuracy)
    explanation_accuracies_inv.append(accuracy)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_2)

results_tree = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_tree.to_csv(os.path.join(results_dir, 'results_tree.csv'))
print(results_tree)

## Relu NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_val = torch.tensor(dataset.attributes[val_data.indices])
    y_val = torch.tensor(dataset.targets[val_data.indices])
    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 1e-5
    hidden_neurons = [200, 100]

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)
    print(f"Training Relu NN...")
    model = XReluNN(n_classes=dataset.n_classes, n_features=n_features,
                    hidden_neurons=hidden_neurons, loss=loss, l1_weight=l1_weight)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=l_r,
                        metric=metric, lr_scheduler=lr_scheduler, device=device, save=False)
    print(results)
    accuracy = model.evaluate(test_data)
    print("Test Model accuracy", accuracy)

    formula, elapsed_time = model.get_global_explanation(x_val, y_val, class_to_explain[0], simplify=True,
                                                         topk_explanations=top_k_explanations,
                                                         concept_names=concept_names, return_time=True)
    exp_accuracy, _ = test_multi_class_explanation(formula, class_to_explain[0], x_test, y_test,
                                                   metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[0]} <-> {formula}")
    print("Elapsed time", elapsed_time)
    print("Explanation accuracy", exp_accuracy)

    formula_2, elapsed_time_2 = model.get_global_explanation(x_val, y_val, class_to_explain[1], simplify=True,
                                                             topk_explanations=top_k_explanations,
                                                             concept_names=concept_names, return_time=True)
    exp_accuracy_2, _ = test_multi_class_explanation(formula_2, class_to_explain[1], x_test, y_test,
                                                     metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[1]} <-> {formula_2}")
    print("Elapsed time", elapsed_time_2)
    print("Explanation accuracy", exp_accuracy_2)

    methods.append("Relu")
    splits.append(seed)
    explanations.append(formula)
    explanations_inv.append(formula_2)
    model_accuracies.append(accuracy)
    explanation_accuracies.append(exp_accuracy)
    explanation_accuracies_inv.append(exp_accuracy_2)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_2)

results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_relu = results[results['method'] == "Relu"]
results_relu.to_csv(os.path.join(results_dir, 'results_relu.csv'))
print(results_relu)

## PSI NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 5e-6
    hidden_neurons = [10, 5]
    fan_in = 3
    lr_psi = 0.01

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)

    print("Training Psi NN...")
    model = PsiNetwork(dataset.n_classes, n_features, hidden_neurons, loss,
                       l1_weight, fan_in=fan_in)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=lr_psi,
                        metric=metric, lr_scheduler=lr_scheduler, device=device, save=False)
    print(results)
    accuracy = model.evaluate(test_data, metric=metric)
    print("Test model accuracy", accuracy)

    formula, elapsed_time = model.get_global_explanation(class_to_explain[0], concept_names,
                                                         simplify=True, return_time=True)
    exp_accuracy, _ = test_multi_class_explanation(formula, class_to_explain[0], x_test, y_test,
                                                   metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[0]} <-> {formula}")
    print("Elapsed time", elapsed_time)
    print("Explanation accuracy", exp_accuracy)

    formula_2, elapsed_time_2 = model.get_global_explanation(class_to_explain[1], concept_names,
                                                             simplify=True, return_time=True)
    exp_accuracy_2, _ = test_multi_class_explanation(formula_2, class_to_explain[1], x_test, y_test,
                                                     metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[1]} <-> {formula_2}")
    print("Elapsed time", elapsed_time_2)
    print("Explanation accuracy", exp_accuracy_2)

    methods.append("Psi")
    splits.append(seed)
    explanations.append(formula)
    explanations_inv.append(formula_2)
    model_accuracies.append(accuracy)
    explanation_accuracies.append(exp_accuracy)
    explanation_accuracies_inv.append(exp_accuracy_2)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time)

results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_psi = results[results['method'] == "Psi"]
results_psi.to_csv(os.path.join(results_dir, 'results_psi.csv'))
print(results_psi)

Seed 0
[    1     3     4 ... 11785 11786 11787]
Training Psi NN...
Epoch: 1/20, Loss: 5.328, Tr_acc: 0.0, Val_acc: 0.0, best_e: -1
Epoch: 2/20, Loss: 5.304, Tr_acc: 0.6, Val_acc: 1.6, best_e: -1
Epoch: 3/20, Loss: 5.189, Tr_acc: 4.0, Val_acc: 8.3, best_e: -1
Epoch: 4/20, Loss: 4.313, Tr_acc: 33.0, Val_acc: 72.0, best_e: -1
Epoch: 5/20, Loss: 1.656, Tr_acc: 86.2, Val_acc: 91.0, best_e: -1
Epoch: 6/20, Loss: 0.850, Tr_acc: 92.2, Val_acc: 92.9, best_e: -1
Epoch: 7/20, Loss: 0.728, Tr_acc: 92.8, Val_acc: 94.0, best_e: -1
Epoch: 8/20, Loss: 0.656, Tr_acc: 93.2, Val_acc: 93.0, best_e: -1
Epoch: 9/20, Loss: 0.633, Tr_acc: 93.2, Val_acc: 93.3, best_e: -1
Epoch: 10/20, Loss: 0.584, Tr_acc: 93.6, Val_acc: 94.0, best_e: -1
Epoch: 11/20, Loss: 3.636, Tr_acc: 28.1, Val_acc: 33.1, best_e: -1
Epoch: 12/20, Loss: 2.681, Tr_acc: 43.0, Val_acc: 47.3, best_e: 12
Epoch: 13/20, Loss: 2.266, Tr_acc: 49.8, Val_acc: 52.4, best_e: 13
Epoch: 14/20, Loss: 2.062, Tr_acc: 53.4, Val_acc: 51.9, best_e: 13
Epoch: 15

## Mu NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_val = torch.tensor(dataset.attributes[val_data.indices])
    y_val = torch.tensor(dataset.targets[val_data.indices])
    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 1e-3
    hidden_neurons = [10, 5]

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)

    print("Training General NN...")
    model = XGeneralNN(n_classes=dataset.n_classes, n_features=n_features, hidden_neurons=hidden_neurons,
                       loss=loss, l1_weight=l1_weight)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=l_r, metric=metric,
                        lr_scheduler=lr_scheduler, device=device, save=False)
    print(results)
    accuracy = model.evaluate(test_data, metric=metric)
    print("Test model accuracy", accuracy)

    formula, elapsed_time = model.get_global_explanation(x_val, y_val, class_to_explain[0], simplify=True,
                                                         topk_explanations=top_k_explanations,
                                                         concept_names=concept_names, return_time=True)
    exp_accuracy, _ = test_multi_class_explanation(formula, class_to_explain[0], x_test, y_test,
                                                   metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[0]} <-> {formula}")
    print("Elapsed time", elapsed_time)
    print("Explanation accuracy", exp_accuracy)

    formula_2, elapsed_time_2 = model.get_global_explanation(x_val, y_val, class_to_explain[1], simplify=True,
                                                             topk_explanations=top_k_explanations,
                                                             concept_names=concept_names, return_time=True)
    exp_accuracy_2, _ = test_multi_class_explanation(formula_2, class_to_explain[1], x_test, y_test,
                                                     metric=metric, concept_names=concept_names)
    print(f"{class_to_explain_names[1]} <-> {formula_2}")
    print("Elapsed time", elapsed_time_2)
    print("Explanation accuracy", exp_accuracy_2)

    methods.append("General")
    splits.append(seed)
    explanations.append(formula)
    explanations_inv.append(formula_2)
    model_accuracies.append(accuracy)
    explanation_accuracies.append(exp_accuracy)
    explanation_accuracies_inv.append(exp_accuracy_2)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_2)

In [ ]:
results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_general = results[results['method'] == "General"]
results_general.to_csv(os.path.join(results_dir, 'results_general.csv'))
results.to_csv(os.path.join(results_dir, 'results.csv'))
print(results)

# Summary

In [ ]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# general
results_general = results[results['method'] == "General"]
df_mean = results_general[cols].mean()
df_sem = results_general[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'general'

# # lime
# df_mean = results_lime[cols].mean()
# df_sem = results_lime[cols].sem()
# df_mean.columns = mean_cols
# df_sem.columns = sem_cols
# summary_lime = pd.concat([df_mean, df_sem])
# summary_lime.name = 'lime'

# relu
results_relu = results[results['method'] == "Relu"]
df_mean = results_relu[cols].mean()
df_sem = results_relu[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'relu'

# psi
results_psi = results[results['method'] == "Psi"]
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
results_tree = results[results['method'] == "Tree"]
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning,
                     #                      summary_lime,
                     summary_weights,
                     summary_psi,
                     summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
print(summary)

In [ ]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))

